# Map with stations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import json
import folium
e = json.load(open('world-countries.json','r')) #Load json with world countries data (shapefiles)
json.dump(e['features'][73], open('india.json','w')) #Obtain outlined map of India


stations_coords = pd.read_csv('station_coords.csv', sep=';') #Load dataset with coordinates of stations
stations_coords_filtered = stations_coords.loc[stations_coords.Lat != '-']

folium_map = folium.Map(width = '60%',height=800,location=[20, 77], #Initialize map
                        zoom_start=5,
                        tiles="Stamen Terrain",min_lat=7, max_lat=35, min_lon=73, max_lon=90)
for x in stations_coords_filtered.iterrows(): #Plot a circle per station of the dataset
    name = x[0]
    lat, lon = x[1]['Lat'], x[1]['Long']
    folium.CircleMarker([lat, lon], radius=5, color='#000000',fill_color='#D3D3D3' , fill_opacity=1).add_to(folium_map)

folium.GeoJson('india.json').add_to(folium_map) #Add India map

folium_map.save("map.html") #Save map

# Feature engineering

In [ ]:
stations_hour = pd.read_csv('station_hour.csv') #Load hourly measurements of stations
stations_info = pd.read_csv('stations.csv') #Load dataset with stations' information

In [ ]:
full_stations = pd.DataFrame() #Initialize dataset

for stat in uni_stations: #Loop through all stations
    station_id = stations_info.loc[stations_info.City == stat, 'StationId'] #Subset of data for the specific station
    
    stations_hour_filtered = stations_hour.loc[stations_hour.StationId == station_id.values[0]]
    stations_hour_filtered.reset_index(drop = True, inplace = True)
    
    if stations_hour_filtered.shape[0] == 0: #If the station is empty, go to the next
        next
    
    #Read weather data corresponding to the city where the station is
    weat_stat = pd.read_csv('weather/' + stat.replace(' ', '+') + ',India.csv')[['date_time', 'DewPointC',
                                                                                'cloudcover', 'humidity',
                                                                               'precipMM', 'pressure',
                                                                               'tempC', 'windspeedKmph']]
    weat_stat_final = weat_stat[['date_time']]
    
    variables_list = []
    
    #Shifted weather variables
    
    for period in [24, 48, 72, 168, 336]: #Loop through lags
        aux_weat_stat = weat_stat.drop('date_time', axis = 1).shift(period) #Apply lag
        aux_weat_stat.columns = ['DewPointC_td' + str(period), 'cloudcover_td'  + str(period),
                                 'humidity_td'  + str(period), 'precipMM_td'  + str(period),
                                'pressure_td'  + str(period), 'tempC_td'  + str(period),
                                 'windspeedKmph_td'  + str(period)]
        
        aux_weat_stat.drop(['cloudcover_td'  + str(period), 'precipMM_td'  + str(period),
                           'tempC_td'  + str(period)], axis = 1, inplace = True)
        
        variables_list = variables_list + list(aux_weat_stat.columns)
        
        weat_stat_final = pd.concat((weat_stat_final, aux_weat_stat), axis = 1) #Add to dataset

    #Rolling average of weather variables (except precipitation)
    
    for period in [24, 48, 72, 168, 336]: #Loop through time window widths
        aux_weat_stat = weat_stat.drop(['precipMM', 'date_time'], axis = 1).rolling(period, min_periods = 24).mean() #Compute average on the window
        aux_weat_stat.columns = ['DewPointC_tro' + str(period), 'cloudcover_tro' + str(period),
                                 'humidity_tro' + str(period), 'pressure_tro' + str(period),
                                 'tempC_tro' + str(period), 'windspeedKmph_tro' + str(period)]
        
        aux_weat_stat.drop(['tempC_tro'  + str(period)], axis = 1, inplace = True)
        
        variables_list = variables_list + list(aux_weat_stat.columns)
        
        weat_stat_final = pd.concat((weat_stat_final, aux_weat_stat.shift(24)), axis = 1)

    #Rolling cumulated sum of precipitations

    for period in [24, 48, 72, 168, 336]:
        aux_weat_stat = weat_stat[['precipMM']].rolling(period, min_periods = 24).sum() #Compute cumulative sum in the window
        aux_weat_stat.columns = ['precipMM_tprep' + str(period)]
        
        variables_list = variables_list + list(aux_weat_stat.columns)
        
        weat_stat_final = pd.concat((weat_stat_final, aux_weat_stat.shift(24)), axis = 1)
    
    #Lagged air quality variables
    
    #all compounds: 'PM2.5','PM10','NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene'
    used_compounds = ['PM2.5', 'NOx', 'SO2', 'O3']
    stations_hour_filtered_aux = stations_hour_filtered[['Datetime'] + used_compounds]
    
    stations_hour_filtered_aux_final = stations_hour_filtered_aux[['Datetime']]
    
    for period in [24, 48, 72, 168, 336]:
        aux_stat_filtered = stations_hour_filtered_aux.drop('Datetime', axis = 1).shift(period)
        
        used_compounds_columns = [x + '_td' + str(period) for x in used_compounds]
        aux_stat_filtered.columns = used_compounds_columns
        
        variables_list = variables_list + list(aux_stat_filtered.columns)
        
        stations_hour_filtered_aux_final = pd.concat((stations_hour_filtered_aux_final, aux_stat_filtered), axis = 1)

    #Rolling average of weather variables (except precipitation)
    
    for period in [24, 48, 72, 168, 336]:
        aux_stat_filtered = stations_hour_filtered_aux.drop(['Datetime'], axis = 1).rolling(period, min_periods = 24).mean()
        used_compounds_columns = [x + '_tro' + str(period) for x in used_compounds]
        aux_stat_filtered.columns = used_compounds_columns
        
        variables_list = variables_list + list(aux_stat_filtered.columns)
        stations_hour_filtered_aux_final = pd.concat((stations_hour_filtered_aux_final, aux_stat_filtered.shift(24)), axis = 1)

    stations_hour_filtered_weather = stations_hour_filtered.merge(weat_stat_final, how = 'inner',
                                                                 left_on = 'Datetime',
                                                                 right_on = 'date_time')
    
    stations_hour_filtered_weather = stations_hour_filtered_weather.merge(stations_hour_filtered_aux_final, how = 'inner',
                                                                 left_on = 'Datetime',
                                                                 right_on = 'Datetime')
    
    stations_hour_filtered_weather['Datetime'] = pd.to_datetime(stations_hour_filtered_weather['Datetime'])
    
    stations_hour_filtered_weather['DoW_sin'] = np.sin(2*np.pi*stations_hour_filtered_weather['Datetime'].dt.dayofweek/7)
    stations_hour_filtered_weather['DoW_cos'] = np.cos(2*np.pi*stations_hour_filtered_weather['Datetime'].dt.dayofweek/7)

    #stations_hour_filtered_weather['Mon'] = stations_hour_filtered_weather['DoW'].apply(lambda x: int(x == 0))
    #stations_hour_filtered_weather['Tue'] = stations_hour_filtered_weather['DoW'].apply(lambda x: int(x == 1))
    #stations_hour_filtered_weather['Wed'] = stations_hour_filtered_weather['DoW'].apply(lambda x: int(x == 2))
    #stations_hour_filtered_weather['Thu'] = stations_hour_filtered_weather['DoW'].apply(lambda x: int(x == 3))
    #stations_hour_filtered_weather['Fri'] = stations_hour_filtered_weather['DoW'].apply(lambda x: int(x == 4))
    #stations_hour_filtered_weather['Sat'] = stations_hour_filtered_weather['DoW'].apply(lambda x: int(x == 5))
    #stations_hour_filtered_weather['Sun'] = stations_hour_filtered_weather['DoW'].apply(lambda x: int(x == 6))
    
    #stations_hour_filtered_weather.drop('DoW', inplace = True, axis = 1)
    
    #variables_list = variables_list + ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat','Sun']
    
    variables_list = variables_list + ['DoW_sin', 'DoW_cos']
    
    stations_hour_filtered_weather['Month_sin'] = np.sin(2*np.pi*stations_hour_filtered_weather['Datetime'].dt.month/12)
    stations_hour_filtered_weather['Month_cos'] = np.cos(2*np.pi*stations_hour_filtered_weather['Datetime'].dt.month/12)

    #stations_hour_filtered_weather['Jan'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 1))
    #stations_hour_filtered_weather['Feb'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 2))
    #stations_hour_filtered_weather['Mar'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 3))
    #stations_hour_filtered_weather['Apr'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 4))
    #stations_hour_filtered_weather['May'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 5))
    #stations_hour_filtered_weather['Jun'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 6))
    #stations_hour_filtered_weather['Jul'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 7))
    #stations_hour_filtered_weather['Aug'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 8))
    #stations_hour_filtered_weather['Sep'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 9))
    #stations_hour_filtered_weather['Oct'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 10))
    #stations_hour_filtered_weather['Nov'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 11))
    #stations_hour_filtered_weather['Dec'] = stations_hour_filtered_weather['Month'].apply(lambda x: int(x == 12))
    #
    #stations_hour_filtered_weather.drop('Month', inplace = True, axis = 1)
    
    variables_list = variables_list + ['Month_sin', 'Month_cos']
    
    stations_hour_filtered_weather['Hour_sin'] = np.sin(2*np.pi*stations_hour_filtered_weather['Datetime'].dt.hour/24)
    stations_hour_filtered_weather['Hour_cos'] = np.cos(2*np.pi*stations_hour_filtered_weather['Datetime'].dt.hour/24)

    variables_list = variables_list + ['Hour_sin', 'Hour_cos']
    
    full_stations = pd.concat((full_stations, stations_hour_filtered_weather), axis = 0)

In [ ]:
full_stations[['PM2.5'] + variables_list].dropna().corr().style.background_gradient(cmap='coolwarm')

In [ ]:
final_data.groupby('StationId').count()

In [ ]:
final_data = full_stations[['StationId', 'date_time', 'PM2.5'] + variables_list].dropna()
final_data.to_csv('final_pm25_3d_2w.csv', index = False) #Save Dataset

## Preprocessing

### Weather

In [ ]:
stations_info = pd.read_csv('stations.csv')
uni_stations = list(stations_info.City.unique())

In [ ]:
weather_data = pd.DataFrame()

for stat in uni_stations:
    weat_stat = pd.read_csv('weather/' + stat.replace(' ', '+') + ',India.csv')[['date_time', 'DewPointC',
                                                                                'cloudcover', 'humidity',
                                                                               'precipMM', 'pressure',
                                                                               'tempC', 'windspeedKmph']]
    
    weather_data = pd.concat((weather_data, weat_stat), axis = 0)

In [ ]:
weather_data.describe()

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, sharey  = True)

ax1.hist(weather_data.DewPointC,20, density = True)
ax2.hist(weather_data.cloudcover,20, density = True)
ax3.hist(weather_data.humidity,20, density = True)
ax4.hist(weather_data.pressure,20, density = True)
ax5.hist(weather_data.tempC,20, density = True)
ax6.hist(weather_data.windspeedKmph,20, density = True)

plt.show()

### Particulate matter data

In [ ]:
stations_hour = pd.read_csv('station_hour.csv')

In [ ]:
stations_hour.columns

In [ ]:
stations_hour.describe()

In [ ]:
sam = stations_hour.sample(1000000)

fig, (ax1, ax2) = plt.subplots(1, 2, sharey  = True)

ax2.hist(sam['O3'].dropna(), 40, density = True)

plt.show()

# Modeling

## Feature selection

In [ ]:
import xgboost as xgb
import pandas as pd

In [ ]:
data = pd.read_csv('final_pm25_3d_2w.csv')

In [ ]:
x_vars = list(set(list(data.columns)) - set(['StationId', 'date_time', 'PM2.5']))
y_var = ['PM2.5']

In [ ]:
from collections import defaultdict

vars_cats = defaultdict(list)
for key, value in [(x[0:3].split('_')[0], x) for x in x_vars]:
    vars_cats[key].append(value)

In [ ]:
vars_cats.keys()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

x = data.loc[:, x_vars]
y = data.loc[:, y_var]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=14)
    
# define random forest classifier
forest = RandomForestRegressor(n_jobs=-1, max_depth=5, verbose = 0)
forest.fit(x_train, y_train)

In [ ]:
from boruta import BorutaPy

# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2, random_state=14)

# find all relevant features
feat_selector.fit(x_train.values, y_train.values.ravel())

In [ ]:
[x_vars[i] for i in range(len(x_vars)) if feat_selector.support_[i] == True]

In [ ]:
feat_selector.ranking_

In [ ]:
feature_ranks = list(zip(x_train.columns, 
                         feat_selector.ranking_, 
                         feat_selector.support_))

# iterate through and print out the results
for feat in feature_ranks:
    print('Feature: {:<25} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))

In [ ]:
data.columns

In [ ]:
data_n = set(data.columns)

new_data = data[list(data_n - set(['Hour_sin', 'Hour_cos', 'DoW_sin', 'DoW_cos', 'Month_sin', 
                                   'O3_td24','O3_td48','O3_td72','O3_td168','O3_td336',
                                  'SO2_td24','SO2_td48','SO2_td72','SO2_td168','SO2_td336']))]

In [ ]:
new_data.to_csv('final_pm25_3days_2w_fs.csv', index = False)

## Modeling

In [ ]:
import xgboost as xgb
import pandas as pd
import sklearn.metrics

In [ ]:
data = pd.read_csv('final_pm25_fs.csv')

In [ ]:
X = data[list(set(data.columns) - set(['date_time', 'StationId', 'PM2.5']))]
y = data[['PM2.5']]

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [ ]:
data_train = data.loc[data.date_time < '2019-07-01']
data_val = data.loc[(data.date_time >= '2019-07-01') & (data.date_time < '2020-01-01')]
data_test = data.loc[(data.date_time >= '2020-01-01')]

X_train = data_train[list(set(data.columns) - set(['date_time', 'StationId', 'PM2.5']))]
y_train = data_train[['PM2.5']]

X_val = data_val[list(set(data.columns) - set(['date_time', 'StationId', 'PM2.5']))]
y_val = data_val[['PM2.5']]

X_test = data_test[list(set(data.columns) - set(['date_time', 'StationId', 'PM2.5']))]
y_test = data_test[['PM2.5']]

### Ordinary Least Squares

#### Raw

In [ ]:
from sklearn import linear_model

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

In [ ]:
predictions_tr = reg.predict(X_train)
predictions_test = reg.predict(np.concatenate((X_test, X_val), axis = 0))

In [ ]:
sklearn.metrics.mean_absolute_error(np.concatenate((y_test, y_val), axis = 0), predictions_test)

In [ ]:
sklearn.metrics.mean_absolute_percentage_error(np.concatenate((y_test, y_val), axis = 0), predictions_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(np.concatenate((y_test, y_val), axis = 0), predictions_test))

In [ ]:
sklearn.metrics.r2_score(np.concatenate((y_test, y_val), axis = 0), predictions_test)

#### Normalizer

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
transformer = Normalizer().fit(X_train)

In [ ]:
transformed_X_train = transformer.transform(X_train)

transformed_X_val = transformer.transform(X_val)

transformed_X_test = transformer.transform(X_test)

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(transformed_X_train, y_train)

In [ ]:
predictions_tr = reg.predict(transformed_X_train)
predictions_test = reg.predict(np.concatenate((transformed_X_test, transformed_X_val), axis = 0))

In [ ]:
sklearn.metrics.mean_absolute_error(np.concatenate((y_test, y_val), axis = 0), predictions_test)

In [ ]:
sklearn.metrics.mean_absolute_percentage_error(np.concatenate((y_test, y_val), axis = 0), predictions_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(np.concatenate((y_test, y_val), axis = 0), predictions_test))

In [ ]:
sklearn.metrics.r2_score(np.concatenate((y_test, y_val), axis = 0), predictions_test)

#### Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
transformer = StandardScaler().fit(X_train)

In [ ]:
transformed_X_train = transformer.transform(X_train)

transformed_X_val = transformer.transform(X_val)

transformed_X_test = transformer.transform(X_test)

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(transformed_X_train, y_train)

In [ ]:
predictions_tr = reg.predict(transformed_X_train)
predictions_test = reg.predict(np.concatenate((transformed_X_test, transformed_X_val), axis = 0))

In [ ]:
sklearn.metrics.mean_absolute_error(np.concatenate((y_test, y_val), axis = 0), predictions_test)

In [ ]:
sklearn.metrics.mean_absolute_percentage_error(np.concatenate((y_test, y_val), axis = 0), predictions_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(np.concatenate((y_test, y_val), axis = 0), predictions_test))

In [ ]:
sklearn.metrics.r2_score(np.concatenate((y_test, y_val), axis = 0), predictions_test)

### Ridge Regression

#### Standarised

In [ ]:
import numpy as np
from sklearn import linear_model

In [ ]:
best_alpha_iter = -300
best_error = 10000
for alpha_iter in np.logspace(-4, 0, 500):
    reg = linear_model.Ridge(alpha = alpha_iter, normalize = True)
    reg.fit(X_train, y_train)
    
    predictions_val = reg.predict(X_val)
    predictions_error = sklearn.metrics.mean_squared_error(y_val, predictions_val)
    
    if predictions_error < best_error:
        best_error = predictions_error
        best_alpha_iter = alpha_iter

In [ ]:
reg = linear_model.Ridge(alpha = best_alpha_iter, normalize = True)
reg.fit(X_train, y_train)

In [ ]:
predictions_tr = reg.predict(X_train)
predictions_test = reg.predict(X_test)

In [ ]:
sklearn.metrics.mean_absolute_error(y_test, predictions_test)

In [ ]:
sklearn.metrics.mean_absolute_percentage_error(y_test, predictions_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test, predictions_test))

In [ ]:
sklearn.metrics.r2_score(y_test, predictions_test)

#### Normalize

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
transformer = Normalizer().fit(X_train)

In [ ]:
transformed_X_train = transformer.transform(X_train)

transformed_X_val = transformer.transform(X_val)

transformed_X_test = transformer.transform(X_test)

In [ ]:
best_alpha_iter = -300
best_error = 10000
for alpha_iter in np.logspace(-4, 0, 500):
    reg = linear_model.Ridge(alpha = alpha_iter, normalize = True)
    reg.fit(transformed_X_train, y_train)
    
    predictions_val = reg.predict(transformed_X_val)
    predictions_error = sklearn.metrics.mean_squared_error(y_val, predictions_val)
    
    if predictions_error < best_error:
        best_error = predictions_error
        best_alpha_iter = alpha_iter

In [ ]:
reg = linear_model.Ridge(alpha = best_alpha_iter)
reg.fit(transformed_X_train, y_train)

In [ ]:
predictions_tr = reg.predict(transformed_X_train)
predictions_test = reg.predict(transformed_X_test)

In [ ]:
sklearn.metrics.mean_absolute_error(y_test, predictions_test)

In [ ]:
sklearn.metrics.mean_absolute_percentage_error(y_test, predictions_test)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test, predictions_test))

In [ ]:
sklearn.metrics.r2_score(y_test, predictions_test)

### SVM

In [ ]:
from skopt import space
from skopt import gp_minimize

In [ ]:
# define the space of hyperparameters to search
search_space = list()
search_space.append(space.space.Real(1e-6, 100.0, 'log-uniform', name='C'))
search_space.append(space.space.Categorical(['linear', 'poly', 'rbf', 'sigmoid'], name='kernel'))
search_space.append(space.space.Integer(1, 5, name='degree'))
search_space.append(space.space.Real(1e-6, 100.0, 'log-uniform', name='gamma'))

In [ ]:
# define the function used to evaluate a given configuration
from skopt.utils import use_named_args
from sklearn.svm import SVR
import sklearn
from sklearn.preprocessing import StandardScaler

X_train_sampled = X_train
y_train_sampled = y_train

X_val_sampled = X_val
y_val_sampled = y_val

transformer = StandardScaler().fit(X_train_sampled)

transformed_X_train = transformer.transform(X_train_sampled)
transformed_X_val = transformer.transform(X_val_sampled)
transformed_X_test = transformer.transform(X_test)

@use_named_args(search_space)
def evaluate_model(**params):
    # configure the model with specific hyperparameters
    model = SVR()
    model.set_params(**params)
    # calculate 5-fold cross validation
    model.fit(transformed_X_train, y_train_sampled.to_numpy().ravel())
    predictions = model.predict(transformed_X_val)
    result = sklearn.metrics.mean_squared_error(y_val_sampled, predictions)
    # convert from a maximizing score to a minimizing score
    return result

In [ ]:
result = gp_minimize(evaluate_model, search_space, verbose = True)

In [ ]:
result.x #without normalization

In [ ]:
model = SVR()
model.set_params(kernel = 'poly', degree = 5, gamma = 1e-06, C = 100)
# calculate 5-fold cross validation
model.fit(transformed_X_train, y_train_sampled.to_numpy().ravel())
predictions = model.predict(transformed_X_val)
sklearn.metrics.mean_absolute_error(y_val_sampled, predictions)

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.kernel_approximation import Nystroem

svm_params = {'C': 3.42860186,
                 'gamma': 0.014033215,
                  'ncomp': 352,
                 'kernel': 'rbf'}
    
model = LinearSVR(random_state = 42, max_iter = 5000, C = svm_params['C'], dual = False, loss = 'squared_epsilon_insensitive')

feature_map_nystroem = Nystroem(gamma= svm_params['gamma'],
                                kernel = svm_params['kernel'],
                             random_state=1,
                             n_components= svm_params['ncomp'])

data_train_transformed = feature_map_nystroem.fit_transform(np.concatenate((transformed_X_train, transformed_X_val), axis = 0))

model.fit(data_train_transformed, np.concatenate((y_train_sampled.to_numpy().ravel(), y_val_sampled.to_numpy().ravel()), axis = 0))

data_test_transformed = feature_map_nystroem.transform(transformed_X_test)
predictions = model.predict(data_test_transformed)
predictions_error = sklearn.metrics.mean_squared_error(y_test, predictions)

In [ ]:
sklearn.metrics.mean_absolute_error(y_test, predictions)

In [ ]:
sklearn.metrics.r2_score(y_test, predictions)

In [ ]:
np.sqrt(sklearn.metrics.mean_squared_error(y_test, predictions))

In [ ]:
sklearn.metrics.mean_absolute_percentage_error(y_test, predictions)

### Random Forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(30, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [ ]:
import random
from sklearn.ensemble import RandomForestRegressor

best_solution = 10000

X_train_sample = X_train[0:20000]
y_train_sample = y_train[0:20000]

for i in range(0,100):
    
    rf_params = {'n_estimators': random.choice(n_estimators),
                 'max_features':random.choice(max_features),
                 'max_depth': random.choice(max_depth),
                 'min_samples_split': random.choice(min_samples_split),
                 'min_samples_leaf': random.choice(min_samples_leaf),
                 'bootstrap': random.choice(bootstrap)
    }

    print(rf_params)
    
    rf = RandomForestRegressor(random_state = 42, n_jobs = -1)
    rf.set_params(**rf_params)
    rf.fit(X_train_sample, y_train_sample.to_numpy().ravel())
    
    predictions = rf.predict(X_val)
    
    predictions_error = sklearn.metrics.mean_absolute_error(y_val, predictions)

    if predictions_error < best_solution:
        best_params = rf_params
        best_solution = predictions_error
        
        print(best_solution)
    
    

In [ ]:
rf_params = {'n_estimators': 500,
                 'max_features': 'sqrt',
                 'max_depth': 60,
                 'min_samples_split': 2,
                 'min_samples_leaf': 1,
                 'bootstrap': False
    }
    
rf = RandomForestRegressor(random_state = 42, n_jobs = -1)
rf.set_params(**rf_params)
rf.fit(np.concatenate((X_train, X_val), axis = 0), np.concatenate((y_train.to_numpy().ravel(), y_val.to_numpy().ravel()), axis = 0))



In [ ]:
rf.feature_importances_

In [ ]:
predictions = rf.predict(X_test)

predictions_error = sklearn.metrics.mean_squared_error(y_test, predictions)

sklearn.metrics.mean_squared_error(y_test, predictions)

In [ ]:
sklearn.metrics.r2_score(y_test, predictions)

In [ ]:
np.mean(np.abs(y_test.to_numpy().ravel() - predictions)/y_test.to_numpy().ravel())

### XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(data = X_train, label = y_train)
dval = xgb.DMatrix(data = X_val, label = y_val)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [ ]:
xgb_params = {'eta': 0.05,
 'max_depth': 50,
 'n_estimators': 1000,
 'min_child_weight': 4,
 'gamma': 0.2,
 'subsample': 0.8,
 'colsample_bytree': 0.6,
 'lambda': 100,
 'alpha': 0.1,
 'objective': 'reg:squarederror',
 'seed': 42,
 'eval_metric': 'rmse'}

In [ ]:
bst = xgb.train(xgb_params, dtrain, 1000, [(dval, 'val')], early_stopping_rounds = 5, maximize = False)

In [ ]:
predictions = bst.predict(dtest)

In [ ]:
predictions_error = sklearn.metrics.mean_squared_error(y_test, predictions)

In [ ]:
np.mean(np.abs(y_test.to_numpy().ravel() - predictions) / y_test.to_numpy().ravel())

In [ ]:
names_list = ['PM2.5', 'Pressure', 'DewPoint', 'Humidity', 'WindSpeed', 'NOx', 'O3', 'CloudCover',
              'Month', 'SO2', 'Precipitations','Temperature', 'CO', 'Hour', 'DayWeek']
var_list = [(1, 0.01), (1, 0.01), (1.05, 0.16), (1.9, 0.74), (2.35, 0.96), (2.4, 0.56),
           (3.6, 0.62), (4.9, 1.94), (5.45, 1.99), (6.75, 1.72), (7, 1.8), (11.85,1.69),
           (13.7, 1.9), (15,2.27), (15.45,2)]
distrs = [np.random.normal(x, y,1000) for (x,y) in var_list]

plt.figure(figsize = (10,5))
plt.boxplot(distrs, vert = False, whis = 1.2, showfliers = False, labels = names_list)
plt.xlim([0, 22])
plt.xlabel('Importance rank')
plt.xticks(np.arange(1, 21+1, 1.0))

plt.show()

## General VS Individual

In [ ]:
unique_stations = np.unique(data.StationId)

In [ ]:
unique_stations

In [ ]:
mse = []
r2 = []
mae = []
nmae = []
predicts_list = []
true_labels = []

for station in unique_stations:
    data_aux = data.loc[data.StationId == station]
    X = data_aux[list(set(data_aux.columns) - set(['date_time', 'StationId', 'PM2.5']))]
    
    if X.shape[0] > 1000 and station != 'TN001' and station != 'KA002':
        y = data_aux[['PM2.5']]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

        rf_params = {'n_estimators': 500,
                 'max_features': 'sqrt',
                 'max_depth': 60,
                 'min_samples_split': 2,
                 'min_samples_leaf': 1,
                 'bootstrap': False
        }

        rf = RandomForestRegressor(random_state = 42, n_jobs = -1)
        rf.set_params(**rf_params)
        rf.fit(np.concatenate((X_train, X_val), axis = 0), np.concatenate((y_train.to_numpy().ravel(), y_val.to_numpy().ravel()), axis = 0))

        predictions = rf.predict(X_test)

        #dtrain = xgb.DMatrix(data = X_train, label = y_train)
        #dval = xgb.DMatrix(data = X_val, label = y_val)
        #dtest = xgb.DMatrix(data = X_test, label = y_test)
    #
    #
        #xgb_params = {'eta': 0.05,
        #     'max_depth': 50,
        #     'n_estimators': 1000,
        #     'min_child_weight': 4,
        #     'gamma': 0.2,
        #     'subsample': 0.8,
        #     'colsample_bytree': 0.6,
        #     'lambda': 100,
        #     'alpha': 0.1,
        #     'objective': 'reg:squarederror',
        #     'seed': 42,
        #     'eval_metric': 'rmse'}
    #
        #bst = xgb.train(xgb_params, dtrain, 1000, [(dval, 'val')], early_stopping_rounds = 5, maximize = False, verbose_eval = False)
    #
        #predictions = bst.predict(dtest)

        mse.append(sklearn.metrics.mean_squared_error(y_test, predictions))
        r2.append(sklearn.metrics.r2_score(y_test, predictions))
        mae.append(sklearn.metrics.mean_absolute_error(y_test, predictions))

        nmae.append(np.mean(np.abs(y_test.to_numpy().ravel() - predictions) / y_test.to_numpy().ravel()))

        predicts_list = predicts_list + list(predictions)
        true_labels = true_labels + [x[0] for x in y_test.values]

        print(mse)
        print(r2)
        print(mae)
        print(nmae)
        print(station)

In [ ]:
sklearn.metrics.mean_absolute_percentage_error(true_labels, predicts_list)

In [ ]:
np.mean(np.abs(np.array(true_labels) - np.array(predicts_list)) / np.array(true_labels))

In [ ]:
stations_data = [data.loc[data.StationId == x].shape[0] for x in unique_stations]

In [ ]:
for i in range(len(unique_stations)):
    print('Station ' + unique_stations[i] + ' (' + str(stations_data[i]) + ' rows) : r2 = ' + str(np.round(np.array(r2[i]), 2)) +
          ' | mae = ' + str(np.round(mae[i], 2)) + ' | rmse = ' + str(np.round(np.sqrt(mse[i]), 2))+
         ' | nmae = ' + str(np.round(nmae[i], 2)))
    
    
    

In [ ]:
np.mean([np.sqrt(mse[i]) for i in range(len(r2))])

In [ ]:
np.mean([nmae[i] for i in range(len(r2))])

In [ ]:
np.mean([np.sqrt(mse[i]) for i in range(len(r2)) if stations_data[i] > 1000 and unique_stations[i] != 'TN001'])

In [ ]:
np.mean([nmae[i] for i in range(len(r2)) if stations_data[i] > 1000 and unique_stations[i] != 'TN001'])